A lire: \
https://brightdata.com/blog/how-tos/web-scraping-with-python#libraries \
https://realpython.com/beautiful-soup-web-scraper-python/#dynamic-websites \
https://selenium-python.readthedocs.io/index.html 


In [5]:
!pip install beautifulsoup4
!pip install requests
!pip install selenium

     |████████████████████████████████| 9.5 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 24.2 MB/s eta 0:00:01
     |████████████████████████████████| 467 kB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 40.2 MB/s eta 0:00:01
ERROR: pie-extended 0.1.2 has requirement numpy<1.24.0, but you'll have numpy 1.24.4 which is incompatible.
ERROR: pie-extended 0.1.2 has requirement requests~=2.25.0, but you'll have requests 2.31.0 which is incompatible.


Tentative de webscrapping des formats txt des 130 résultats avec des occurences d'"Illuminati"

In [8]:
import requests
from bs4 import BeautifulSoup

def scrape_txt_url(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all <a> tags with an href attribute containing ".txt"
        txt_links = soup.find_all('a', href=lambda href: href and href.endswith('.txt'))

        # If there are text file links found, return the first one
        if txt_links:
            return txt_links[0]['href']
        else:
            print("Error: Text file not found.")
            return None
    else:
        print("Error: Unable to fetch data from the website")
        return None

# URL of the specific result page to scrape
url = 'https://chroniclingamerica.loc.gov/lccn/sn88053122/1831-09-30/ed-1/seq-2/#date1=1756&index=0&rows=20&words=Illuminati&searchType=basic&sequence=0&state=&date2=1850&proxtext=Illuminati&y=0&x=0&dateFilterType=yearRange&page=1'

# Scrape the URL of the txt file
txt_url = scrape_txt_url(url)

# Print the txt file URL
if txt_url:
    print(f"Text File URL: {txt_url}")
else:
    print("No text file URL found.")

Error: Text file not found.
No text file URL found.


Tentative avec 1 texte à partir du lien du fichier txt (suffixe ocr.txt)

In [9]:
import requests

def download_txt_file(url, file_path):
    # Send a GET request to download the file
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Write the content to a file
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print("File downloaded successfully.")
    else:
        print("Error: Unable to download the file.")

# URL of the text file to download
url = 'https://chroniclingamerica.loc.gov/lccn/sn83045634/1798-12-10/ed-1/seq-1/ocr.txt'

# File path to save the downloaded file
file_path = 'essai1.txt'

# Download the text file and save it to the specified file path
download_txt_file(url, file_path)

File downloaded successfully.


In [11]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

def download_txt_file(url, file_path):
    # Send a GET request to download the file
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Write the content to a file
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"File '{file_path}' downloaded successfully.")
    else:
        print(f"Error: Unable to download the file from '{url}'.")

def scrape_txt_urls(search_url):
    # Send a GET request to the search results page
    response = requests.get(search_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all <a> tags with an href attribute containing "ocr.txt"
        txt_links = soup.find_all('a', href=lambda href: href and href.endswith('ocr.txt'))
        
        # Extract the URLs from the href attributes
        txt_urls = [link['href'] for link in txt_links]
        
        return txt_urls
    else:
        print(f"Error: Unable to fetch data from the website '{search_url}'.")
        return []

# URL of the search results page
search_url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1850&proxtext=Illuminati&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic'

# Scrape the URLs of the text files from the search results page
txt_urls = scrape_txt_urls(search_url)

# Download each text file
for idx, txt_url in enumerate(txt_urls, start=1):
    # Parse the URL to extract information
    parsed_url = urlparse(txt_url)
    query_params = parse_qs(parsed_url.query)
    
    # Extract newspaper name, location, and date from the query parameters
    newspaper_name = query_params.get('lccn', [''])[0]
    location = query_params.get('state', [''])[0]
    date = query_params.get('date', [''])[0]
    
    # Construct the file name
    file_name = f'{newspaper_name}_{location}_{date}.txt'
    print(f"Download link: {txt_url}, File name: {file_name}")
    
    # Download the text file
    download_txt_file(txt_url, file_name)

Recherches sur structuration des fichiers

In [15]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

def download_txt_file(url, file_path):
    # Send a GET request to download the file
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Write the content to a file
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"File '{file_path}' downloaded successfully.")
    else:
        print(f"Error: Unable to download the file from '{url}'.")

def scrape_txt_urls(search_url):
    # Send a GET request to the search results page
    response = requests.get(search_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all <a> tags with an href attribute containing "ocr.txt"
        txt_links = soup.find_all('a', href=lambda href: href and href.endswith('ocr.txt'))
        
        # Extract the URLs from the href attributes
        txt_urls = [link['href'] for link in txt_links]
        
        return txt_urls
    else:
        print(f"Error: Unable to fetch data from the website '{search_url}'.")
        return []

# URL of the search results page
search_url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1850&proxtext=Illuminati&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic'

# Scrape the URLs of the text files from the search results page
txt_urls = scrape_txt_urls(search_url)

# Print out the retrieved text file URLs
print("Text File URLs:")
for url in txt_urls:
    print(url)

Text File URLs:


Tentative debug

In [17]:
import requests
from bs4 import BeautifulSoup

def scrape_results_page(url):
    # Send a GET request to the search results page
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print out the response content to check if we're getting the HTML content correctly
        print(response.content)
        
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all the result items
        result_items = soup.find_all('div', class_='item')
        
        # Check if any result items were found
        if result_items:
            print("Result items found:")
            for item in result_items:
                print(item)
        else:
            print("No result items found.")
    else:
        print(f"Error: Unable to fetch data from the website '{url}'.")

# URL of the original results page
results_page_url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1850&proxtext=Illuminati&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic'

# Scrape the results page and print the response content and result items
scrape_results_page(results_page_url)

b'<!DOCTYPE html>\n<html>\n  <head profile="http://dublincore.org/documents/2008/08/04/dc-html/">\n    \n    \n    \n      <title>Search Results \xc2\xab Chronicling America \xc2\xab Library of Congress</title>\n      <meta http-equiv="Content-Language" content="en-us" />\n      <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n      <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0"/>\n      <link rel="stylesheet" media="screen" type="text/css" href=/media/css/lochead.06d82f58f829.css />\n      <link rel="stylesheet" media="screen" type="text/css" href=/media/css/locfoot.67de26538680.css />\n      <link rel="preload" as="script" href="/media/js/jquery.min.883bf0ec72cf.js">\n      <link rel="preload" as="script" href="/media/js/lab.min.e842152f94c9.js">\n      \n        <script async src="https://assets.adobedtm.com/f94f5647937d/624e2240e90d/launch-0610ec681aff.min.js"></script>\n      \n    \n    <style>\n    

In [18]:
import requests
from bs4 import BeautifulSoup

def scrape_results_page(url):
    # Send a GET request to the search results page
    response = requests.get(url)
    
    # Check if the request was successful
    if response.ok:
        # Print out the response content to check if we're getting the HTML content correctly
        print(response.content)
        
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all the result items
        result_items = soup.find_all('div', class_='item')
        
        # Check if any result items were found
        if result_items:
            print("Result items found:")
            for item in result_items:
                print(item)
        else:
            print("No result items found.")
    else:
        # Print error message
        print(f"Error: Unable to fetch data from the website '{url}'. Status code: {response.status_code}")

# URL of the original results page
results_page_url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1850&proxtext=Illuminati&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic'

# Scrape the results page and print the response content and result items
scrape_results_page(results_page_url)

b'<!DOCTYPE html>\n<html>\n  <head profile="http://dublincore.org/documents/2008/08/04/dc-html/">\n    \n    \n    \n      <title>Search Results \xc2\xab Chronicling America \xc2\xab Library of Congress</title>\n      <meta http-equiv="Content-Language" content="en-us" />\n      <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n      <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0"/>\n      <link rel="stylesheet" media="screen" type="text/css" href=/media/css/lochead.06d82f58f829.css />\n      <link rel="stylesheet" media="screen" type="text/css" href=/media/css/locfoot.67de26538680.css />\n      <link rel="preload" as="script" href="/media/js/jquery.min.883bf0ec72cf.js">\n      <link rel="preload" as="script" href="/media/js/lab.min.e842152f94c9.js">\n      \n        <script async src="https://assets.adobedtm.com/f94f5647937d/624e2240e90d/launch-0610ec681aff.min.js"></script>\n      \n    \n    <style>\n    

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def scrape_result_page_urls(results_page_url):
    # Send a GET request to the results page
    response = requests.get(results_page_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all the links to individual result pages
        result_page_links = soup.find_all('a', class_='title')
        
        # Extract the URLs from the links
        result_page_urls = [link['href'] for link in result_page_links]
        
        return result_page_urls
    else:
        print(f"Error: Unable to fetch data from the website '{results_page_url}'. Status code: {response.status_code}")
        return []

def download_txt_versions(result_page_urls):
    if not result_page_urls:
        print("No result page URLs found.")
        return
    
    for result_page_url in result_page_urls:
        # Append "ocr.txt/" to the result page URL to get the text version URL
        txt_version_url = urljoin(result_page_url, 'ocr.txt/')
        
        print(f"Downloading text version from: {txt_version_url}")
        
        # Send a GET request to download the text version
        response = requests.get(txt_version_url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Save the text version to a file
            filename = result_page_url.split('/')[-2] + "_ocr.txt"
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded text version: {filename}")
        else:
            print(f"Error: Unable to download text version from '{txt_version_url}'. Status code: {response.status_code}")

# URL of the results page
results_page_url = 'https://chroniclingamerica.loc.gov/search/pages/results/list/?date1=1756&rows=50&searchType=basic&state=&date2=1850&proxtext=Illuminati&y=0&x=0&dateFilterType=yearRange&page=1&sort=relevance'

# Scrape the results page to obtain the URLs of individual result pages
result_page_urls = scrape_result_page_urls(results_page_url)

# Download the text versions from the obtained URLs
download_txt_versions(result_page_urls)

format des liens recherchés 
https://chroniclingamerica.loc.gov/*/*/*/*/*/ocr.txt/#date1=1756&index=1&rows=20&words=Illuminati&searchType=basic&sequence=0&state=&date2=1850&proxtext=Illuminati&y=0&x=0&dateFilterType=yearRange&page=1

inspection du code html des pages web des résultats <link rel= ‘alternate’ type= ‘text/plain” href= ‘/lccn/sn83045634/1798-12-10/ed-1/seq-1/ocr.txt>

Tentative avec une boucle

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def retrieve_txt_files(results_page_url):
    # Send a GET request to the results page
    response = requests.get(results_page_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all links to individual result pages
        result_page_links = soup.find_all('a', class_='title')
        
        # Iterate over each result page link
        for link in result_page_links:
            # Construct the URL for the individual result page
            individual_result_page_url = urljoin(results_page_url, link['href'])
            
            # Retrieve and process the text file from the individual result page
            process_individual_result_page(individual_result_page_url)
    else:
        print(f"Error: Unable to fetch data from the website '{results_page_url}'.")

def process_individual_result_page(individual_result_page_url):
    # Send a GET request to the individual result page
    response = requests.get(individual_result_page_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract necessary information (newspaper identifier, date, edition, sequence number)
        # Example code: identifier = ...
        # Example code: date = ...
        # Example code: edition = ...
        # Example code: sequence_number = ...
        
        # Construct the URL for the text file using the extracted information
        txt_file_url = construct_txt_file_url(individual_result_page_url, identifier, date, edition, sequence_number)
        
        # Download the text file
        download_txt_file(txt_file_url)
    else:
        print(f"Error: Unable to fetch data from the website '{individual_result_page_url}'.")

def construct_txt_file_url(individual_result_page_url, identifier, date, edition, sequence_number):
    # Example implementation:
    # Construct the URL for the text file based on the provided format
    txt_file_url = f"{individual_result_page_url}/ocr.txt"
    return txt_file_url

def download_txt_file(txt_file_url):
    # Send a GET request to download the text file
    response = requests.get(txt_file_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Save the text file
        filename = txt_file_url.split('/')[-2] + "_ocr.txt"
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded text file: {filename}")
    else:
        print(f"Error: Unable to download text file from '{txt_file_url}'.")

# URL of the results page containing links to individual result pages
results_page_url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1850&proxtext=Illuminati&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic'

# Retrieve and process text files from individual result pages
retrieve_txt_files(results_page_url)

La récupération des données s'avérait trop difficile pour moi, pour l'instant. Heureusement, la Librairie du Congrès met à disposition une API, qui va me permettre de récupérer mes données. 

API \
https://github.com/LibraryOfCongress/data-exploration \
Les premières lignes de code suivantes m'ont permis d'apprivoiser les spécificités de l'API en question.


In [ ]:
import requests #import the library that we use to make the http request
r = requests.get("http://chroniclingamerica.loc.gov/search/pages/results/?state=Arizona&format=json").json()
r['itemsPerPage'] #print out the number of items per page:

In [3]:
import requests #import the library that we use to make the http request
r = requests.get("http://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1776&date2=1850&proxtext=Illuminati&dateFilterType=yearRange&format=json").json()
r['itemsPerPage'] #print out the number of items per page:


20

In [7]:
illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?proxtext=Illuminati&format=json').json()

In [8]:
illuminati_search['totalItems']

10469

In [9]:
illuminati_search['items'][0]['url']

'https://chroniclingamerica.loc.gov/lccn/45043535/1901-05-01/ed-1/seq-27.json'

In [40]:
illuminati_search['items'][0]['title']

'The Indian advocate.'

In [42]:
illuminati_search['items'][0]['date']

'19010501'

In [10]:
requests.get('https://chroniclingamerica.loc.gov/lccn/45043535/1901-05-01/ed-1/seq-27/ocr.txt').text

'wmrgit Kxn\'""\' \'i ((vw\nTin: Indian Advocate. 152\nall property belonging to the foundations, abbeys and monas\nteries within their states, and to dispose of it at their discre\ntion in providing public worship (Protestant) and instruc\ntion, in founding useful institutions, and " restoring their own\nfinances."\nIn this way the whole of the temporal possessions of sev\neral bishoprics were disposed of. Unfortunately, most of the\nbishops died during the first fifteen years of the century; their\nchairs remained vacant, and it seemed as if the Church was\ndoomed to the same fate as the old empire which had been\ndissolved in 1806 on the death of Francis II. But the more\ngrass is cut, the more vigorously does it grow.\nIn spite of the diabolical doctrines and the growing\npower of the illuminati, the Church continued. Voltaire,\nKant and Weishaupt had caused men of brain and thought to\nconsider religious matters, and these considerations brought\nmany illustrious converts to the C

In [33]:
pages = requests.get("http://chroniclingamerica.loc.gov/search/pages/results/?proxtext=Illuminati&rows=20&format=json").json()

In [12]:
from collections import Counter

c = Counter()
for page in pages['items']:
    c[page['state'][0]] += 1
    
c.most_common()

[('District of Columbia', 3),
 ('Delaware', 2),
 ('Maine', 2),
 ('Utah', 2),
 ('Minnesota', 2),
 ('Oklahoma', 1),
 ('Connecticut', 1),
 ('Michigan', 1),
 ('Texas', 1),
 ('Maryland', 1),
 ('North Carolina', 1),
 ('Illinois', 1),
 ('Virginia', 1),
 ('New York', 1)]

In [21]:
illuminati_search_1776_1850 = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1776&date2=1850&proxtext=Illuminati&dateFilterType=yearRange&format=json').json()

?state=&date1=1756&date2=1776&proxtext=Illuminati&x=4&y=20&dateFilterType=yearRange&rows=20 \
copie format lien requête à la main

In [22]:
illuminati_search_1776_1850['totalItems'] #on a le bon nombre

130

In [23]:
illuminati_search_1776_1850['items'][0]['url']

'https://chroniclingamerica.loc.gov/lccn/sn88053122/1831-09-30/ed-1/seq-2.json'

In [ ]:
illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?proxtext=Illuminati&format=json').json()
illuminati_search_1776_1850['items'][0]['url']
illuminati_search_1776_1850['items'][0]['title']
illuminati_search_1776_1850['items'][0]['date']
requests.get('https://chroniclingamerica.loc.gov/lccn/sn88053122/1831-09-30/ed-1/seq-2/ocr.txt').text

In [24]:
requests.get('https://chroniclingamerica.loc.gov/lccn/sn88053122/1831-09-30/ed-1/seq-2/ocr.txt').text

'By several Â«rivals at New York, London and of\nParis papers to the 16th of August have been\nreceived. h or our extracts we arc milÂ«. > 0 ; m a\nour correspondents of the American, the Stan- ^\n, dar<l, the Morning Gouner, and the Daily. ^\nAdvertiser.\nFrom the London Sun of Aagart Su.. ^\nMarch of the French ,Â® ; cast II\nportance thicken on usâ\x80\x94thedieol war is cast\nLand a French army of 50 000 men, now\non ns march to assist the King of the Behans\nin repelling the invasion of the Dutch. This ^\nis indeed important news, and must lead *Â° -,\ntesults of great importance. . . ...\nIt appears that on Thursday night, intelli-|\ngence had been received at Paris of the Dutch!\nhaving broken the armistice, and commenced,\nimmediate hostilities, bv the capture of some\nof the Belgian towns. Nearly at the same time\nan express arrived from King Leopold, tlemand\ning the aid of a French army to preserve the\nneutrality of Belgium, and to repel the Dutch.\nThe French King immediat

In [25]:
illuminati_search_1776_1850['items'][0]['ocr_eng']

'By several «rivals at New York, London and of\nParis papers to the 16th of August have been\nreceived. h or our extracts we arc mil«. > 0 ; m a\nour correspondents of the American, the Stan- ^\n, dar<l, the Morning Gouner, and the Daily. ^\nAdvertiser.\nFrom the London Sun of Aagart Su.. ^\nMarch of the French ,® ; cast II\nportance thicken on us—thedieol war is cast\nLand a French army of 50 000 men, now\non ns march to assist the King of the Behans\nin repelling the invasion of the Dutch. This ^\nis indeed important news, and must lead *° -,\ntesults of great importance. . . ...\nIt appears that on Thursday night, intelli-|\ngence had been received at Paris of the Dutch!\nhaving broken the armistice, and commenced,\nimmediate hostilities, bv the capture of some\nof the Belgian towns. Nearly at the same time\nan express arrived from King Leopold, tlemand\ning the aid of a French army to preserve the\nneutrality of Belgium, and to repel the Dutch.\nThe French King immediately convoked

In [45]:
import requests

# Original request to retrieve search results
illuminati_search_1776_1850 = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1776&date2=1850&proxtext=Illuminati&dateFilterType=yearRange&format=json').json()

ocr_files = []

# Loop through each search result and retrieve OCR text
for result in illuminati_search_1776_1850['items']:
    # Extract newspaper name and publication date
    newspaper_name = result['title']
    publication_date = result['date']
    # Create file name by combining newspaper name and publication date
    file_name = f"{newspaper_name}_{publication_date}.txt"
    # Extract necessary information for constructing the OCR text URL
    lccn = result['lccn']
    sequence = result['sequence']
    # Construct OCR text URL
    ocr_url = f'https://chroniclingamerica.loc.gov/lccn/{lccn}/{publication_date.replace("-", "")}/{sequence}/ocr.txt'
    # Request OCR text content from the OCR text URL
    ocr_response = requests.get(ocr_url)
    # Check if request was successful
    if ocr_response.status_code == 200:
        # Extract OCR text from response
        ocr_text = ocr_response.text
        # Append file name and OCR text content to the list of OCR files
        ocr_files.append({file_name: ocr_text})
    else:
        print(f"Failed to retrieve OCR text for {file_name}. Status code: {ocr_response.status_code}, URL: {ocr_url}")

# Print the number of OCR text files retrieved
print("Number of OCR text files retrieved:", len(ocr_files))

Failed to retrieve OCR text for Wilmington expositor._18310930.txt. Status code: 404, URL: https://chroniclingamerica.loc.gov/lccn/sn88053122/18310930/2/ocr.txt
Failed to retrieve OCR text for The gazette. [volume]_17981210.txt. Status code: 404, URL: https://chroniclingamerica.loc.gov/lccn/sn83045634/17981210/1/ocr.txt
Failed to retrieve OCR text for The national intelligencer and Washington advertiser. [volume]_18020915.txt. Status code: 404, URL: https://chroniclingamerica.loc.gov/lccn/sn83045242/18020915/2/ocr.txt
Failed to retrieve OCR text for The recorder, or, Lady's and gentleman's miscellany. [volume]_18020424.txt. Status code: 404, URL: https://chroniclingamerica.loc.gov/lccn/sn84024678/18020424/4/ocr.txt
Failed to retrieve OCR text for Jenks's Portland gazette. [volume]_18020920.txt. Status code: 404, URL: https://chroniclingamerica.loc.gov/lccn/sn83016063/18020920/2/ocr.txt
Failed to retrieve OCR text for American watchman and Delaware advertiser. [volume]_18250510.txt. Sta

In [48]:
import os
import requests

# Perform search for "Illuminati"
illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1776&date2=1850&proxtext=Illuminati&dateFilterType=yearRange&format=json').json()

# Extract necessary information from the first search result
first_result = illuminati_search['items'][0]
title = first_result['title']
date = first_result['date']
ocr_text = first_result['ocr_eng']

# Define file name format
file_name = f"{title}{date}.txt"

# Store OCR text content under the file name
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(ocr_text)

print(f"Stored OCR text for {title} ({date}) as {file_name}")

Stored OCR text for Wilmington expositor. (18310930) as Wilmington expositor.18310930.txt


In [59]:
import os
import requests

# Create a folder named "essai_automatisation_OCR_files" if it doesn't exist
folder_name = "essai_automatisation_OCR_files"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Perform search for "Illuminati"
illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1776&date2=1850&proxtext=Illuminati&dateFilterType=yearRange&format=json').json()

# Loop through each search result
for result in illuminati_search['items']:
    title = result['title']
    date = result['date']
    
    # Check if OCR text is available
    if 'ocr_eng' in result:
        ocr_text = result['ocr_eng']
        
        # Define file name format
        file_name = f"{title}_{date}.txt"
        
        # Define file path
        file_path = os.path.join(folder_name, file_name)
        
        # Store OCR text content under the file name in the folder
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(ocr_text)
        
        print(f"Stored OCR text for {title} ({date}) as {file_path}")
    else:
        print(f"No OCR text available for {title} ({date}), skipping...")

Stored OCR text for Wilmington expositor. (18310930) as essai_automatisation_OCR_files/Wilmington expositor._18310930.txt
Stored OCR text for The gazette. [volume] (17981210) as essai_automatisation_OCR_files/The gazette. [volume]_17981210.txt
Stored OCR text for The national intelligencer and Washington advertiser. [volume] (18020915) as essai_automatisation_OCR_files/The national intelligencer and Washington advertiser. [volume]_18020915.txt
Stored OCR text for The recorder, or, Lady's and gentleman's miscellany. [volume] (18020424) as essai_automatisation_OCR_files/The recorder, or, Lady's and gentleman's miscellany. [volume]_18020424.txt
Stored OCR text for Jenks's Portland gazette. [volume] (18020920) as essai_automatisation_OCR_files/Jenks's Portland gazette. [volume]_18020920.txt
Stored OCR text for American watchman and Delaware advertiser. [volume] (18250510) as essai_automatisation_OCR_files/American watchman and Delaware advertiser. [volume]_18250510.txt
Stored OCR text for 

Il semblerait que seulement 20 des 130 résultats ont des fichiers OCR exploitables. 

In [54]:
illuminati_search_1776_1850['totalItems']

130

In [56]:
illuminati_search_1776_1850['items'][50]['url'] #incohérent after cellule précédente

IndexError: list index out of range

A la main

In [61]:
import os
import requests

# Perform search for "Illuminati"
illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1776&date2=1850&proxtext=Illuminati&dateFilterType=yearRange&format=json').json()


r0 = illuminati_search_1776_1850['items'][0]
title = r0['title']
date = r0['date']
ocr_text = r0['ocr_eng']

# Define file name format
file_name = f"{title}{date}.txt"

# Store OCR text content under the file name
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(ocr_text)

print(f"Stored OCR text for {title} ({date}) as {file_name}")

Stored OCR text for Wilmington expositor. (18310930) as Wilmington expositor.18310930.txt


In [62]:
r1 = illuminati_search_1776_1850['items'][1]
title = r1['title']
date = r1['date']
ocr_text = r1['ocr_eng']

# Define file name format
file_name = f"{title}{date}.txt"

# Store OCR text content under the file name
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(ocr_text)

print(f"Stored OCR text for {title} ({date}) as {file_name}")

Stored OCR text for The gazette. [volume] (17981210) as The gazette. [volume]17981210.txt


In [63]:
r2 = illuminati_search_1776_1850['items'][2]
title = r2['title']
date = r2['date']
ocr_text = r2['ocr_eng']

# Define file name format
file_name = f"{title}{date}.txt"

# Store OCR text content under the file name
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(ocr_text)

print(f"Stored OCR text for {title} ({date}) as {file_name}")

Stored OCR text for The national intelligencer and Washington advertiser. [volume] (18020915) as The national intelligencer and Washington advertiser. [volume]18020915.txt


In [65]:
r20 = illuminati_search_1776_1850['items'][20]
title = r20['title']
date = r20['date']
ocr_text = r20['ocr_eng']

# Define file name format
file_name = f"{title}{date}.txt"

# Store OCR text content under the file name
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(ocr_text)

print(f"Stored OCR text for {title} ({date}) as {file_name}")

IndexError: list index out of range

In [66]:
illuminati_search_1776_1850['totalItems']

130

Problème de pagination, par défaut l'affichage est de 20 résultats par page. On cherche alors à l'augmenter.

In [8]:
#Page 2
import os
import requests
from json.decoder import JSONDecodeError

# Create a folder named "essai_automatisation_OCR_files" if it doesn't exist
folder_name = "essai_automatisation_OCR_files"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

try:
    # Perform search for "Illuminati"
    illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?date1=1776&rows=50&searchType=basic&state=&date2=1850&proxtext=Illuminati&y=13&x=24&dateFilterType=yearRange&page=1&page=2&page=3&format=json').json()    
    # Loop through each search result
    for result in illuminati_search['items']:
        title = result['title']
        date = result['date']
        
        # Check if OCR text is available
        if 'ocr_eng' in result:
            ocr_text = result['ocr_eng']
            
            # Define file name format
            file_name = f"{title}_{date}.txt"
            
            # Define file path
            file_path = os.path.join(folder_name, file_name)
            
            # Store OCR text content under the file name in the folder
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(ocr_text)
            
            print(f"Stored OCR text for {title} ({date}) as {file_path}")
        else:
            print(f"No OCR text available for {title} ({date}), skipping...")
except JSONDecodeError:
    print("Error decoding JSON response. The response may be empty or not in the expected format.")

Stored OCR text for Martinsburg gazette. [volume] (18401119) as essai_automatisation_OCR_files/Martinsburg gazette. [volume]_18401119.txt
Stored OCR text for Staunton spectator, and general advertiser. [volume] (18490509) as essai_automatisation_OCR_files/Staunton spectator, and general advertiser. [volume]_18490509.txt
Stored OCR text for The liberator. [volume] (18420304) as essai_automatisation_OCR_files/The liberator. [volume]_18420304.txt
Stored OCR text for Richmond enquirer. [volume] (18180818) as essai_automatisation_OCR_files/Richmond enquirer. [volume]_18180818.txt
Stored OCR text for The New York herald. [volume] (18500213) as essai_automatisation_OCR_files/The New York herald. [volume]_18500213.txt
Stored OCR text for The New York herald. [volume] (18441012) as essai_automatisation_OCR_files/The New York herald. [volume]_18441012.txt
Stored OCR text for Richmond enquirer. [volume] (18440507) as essai_automatisation_OCR_files/Richmond enquirer. [volume]_18440507.txt
Stored O

In [4]:
    illuminati_search = requests.get('http://chroniclingamerica.loc.gov/search/pages/results/?date1=1776&rows=50&searchType=basic&state=&date2=1850&proxtext=Illuminati&y=13&x=24&dateFilterType=yearRange&page1=1&page2=3&pageFilterType=pageRange&format=json').json()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
#Déjà mieux, 96 OCR récup sur 130 résultats
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from json.decoder import JSONDecodeError

# Create a folder named "essai_automatisation_OCR_files" if it doesn't exist
folder_name = "essai_automatisation_OCR_files_p2"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/?date1=1776&rows=50&searchType=basic&state=&date2=1850&proxtext=Illuminati&y=13&x=24&dateFilterType=yearRange&page={}&format=json'

page_number = 1

# Define retry strategy
retry_strategy = Retry(
    total=3,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)

# Create session with retry mechanism
session = requests.Session()
session.mount("http://", adapter)

try:
    while True:
        # Construct URL for the current page
        url = base_url.format(page_number)
        
        # Perform the request with retry
        response = session.get(url)
        
        # Check if response is successful
        if response.status_code == 200:
            try:
                # Parse JSON response
                illuminati_search = response.json()
                
                # Loop through each search result
                for result in illuminati_search['items']:
                    title = result['title']
                    date = result['date']
                    
                    # Check if OCR text is available
                    if 'ocr_eng' in result:
                        ocr_text = result['ocr_eng']
                        
                        # Define file name format
                        file_name = f"{title}_{date}.txt"
                        
                        # Define file path
                        file_path = os.path.join(folder_name, file_name)
                        
                        # Store OCR text content under the file name in the folder
                        with open(file_path, 'w', encoding='utf-8') as file:
                            file.write(ocr_text)
                        
                        print(f"Stored OCR text for {title} ({date}) as {file_path}")
                    else:
                        print(f"No OCR text available for {title} ({date}), skipping...")
                
                # Move to the next page
                page_number += 1
            except JSONDecodeError:
                print("Error decoding JSON response. The response may be empty or not in the expected format.")
                break
        else:
            print("Request failed with status code:", response.status_code)
            break
except Exception as e:
    print("An error occurred:", e)

Stored OCR text for Wilmington expositor. (18310930) as essai_automatisation_OCR_files_p2/Wilmington expositor._18310930.txt
Stored OCR text for The gazette. [volume] (17981210) as essai_automatisation_OCR_files_p2/The gazette. [volume]_17981210.txt
Stored OCR text for The national intelligencer and Washington advertiser. [volume] (18020915) as essai_automatisation_OCR_files_p2/The national intelligencer and Washington advertiser. [volume]_18020915.txt
Stored OCR text for The recorder, or, Lady's and gentleman's miscellany. [volume] (18020424) as essai_automatisation_OCR_files_p2/The recorder, or, Lady's and gentleman's miscellany. [volume]_18020424.txt
Stored OCR text for Jenks's Portland gazette. [volume] (18020920) as essai_automatisation_OCR_files_p2/Jenks's Portland gazette. [volume]_18020920.txt
Stored OCR text for American watchman and Delaware advertiser. [volume] (18250510) as essai_automatisation_OCR_files_p2/American watchman and Delaware advertiser. [volume]_18250510.txt
St

In [11]:
#Encore mieux, 126/130 résultats récupérés 
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from json.decoder import JSONDecodeError

# Create a folder named "essai_automatisation_OCR_files" if it doesn't exist
folder_name = "essai_automatisation_OCR_files_3"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/?date1=1776&rows=50&searchType=basic&state=&date2=1850&proxtext=Illuminati&y=13&x=24&dateFilterType=yearRange&page={}&format=json'

page_number = 1
pages_to_fetch = 3  # Set the number of pages to fetch

# Define retry strategy
retry_strategy = Retry(
    total=3,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)

# Create session with retry mechanism
session = requests.Session()
session.mount("http://", adapter)

try:
    while page_number <= pages_to_fetch:
        # Construct URL for the current page
        url = base_url.format(page_number)
        
        # Perform the request with retry
        response = session.get(url)
        
        # Check if response is successful
        if response.status_code == 200:
            try:
                # Parse JSON response
                illuminati_search = response.json()
                
                # Loop through each search result
                for result in illuminati_search['items']:
                    title = result['title']
                    date = result['date']
                    
                    # Check if OCR text is available
                    if 'ocr_eng' in result:
                        ocr_text = result['ocr_eng']
                        
                        # Define file name format
                        file_name = f"{title}_{date}.txt"
                        
                        # Define file path
                        file_path = os.path.join(folder_name, file_name)
                        
                        # Store OCR text content under the file name in the folder
                        with open(file_path, 'w', encoding='utf-8') as file:
                            file.write(ocr_text)
                        
                        print(f"Stored OCR text for {title} ({date}) as {file_path}")
                    else:
                        print(f"No OCR text available for {title} ({date}), skipping...")
                
                # Move to the next page
                page_number += 1
            except JSONDecodeError:
                print("Error decoding JSON response. The response may be empty or not in the expected format.")
                break
        else:
            print("Request failed with status code:", response.status_code)
            break
except Exception as e:
    print("An error occurred:", e)

Stored OCR text for Wilmington expositor. (18310930) as essai_automatisation_OCR_files_3/Wilmington expositor._18310930.txt
Stored OCR text for The gazette. [volume] (17981210) as essai_automatisation_OCR_files_3/The gazette. [volume]_17981210.txt
Stored OCR text for The national intelligencer and Washington advertiser. [volume] (18020915) as essai_automatisation_OCR_files_3/The national intelligencer and Washington advertiser. [volume]_18020915.txt
Stored OCR text for The recorder, or, Lady's and gentleman's miscellany. [volume] (18020424) as essai_automatisation_OCR_files_3/The recorder, or, Lady's and gentleman's miscellany. [volume]_18020424.txt
Stored OCR text for Jenks's Portland gazette. [volume] (18020920) as essai_automatisation_OCR_files_3/Jenks's Portland gazette. [volume]_18020920.txt
Stored OCR text for American watchman and Delaware advertiser. [volume] (18250510) as essai_automatisation_OCR_files_3/American watchman and Delaware advertiser. [volume]_18250510.txt
Stored O

Il ne semble pas que cela soit un problème de pagination ici, mais un problème plus complexe qu'il me semble difficile à régler cette année. Pour la taille du corpus donné, il me semble pas trop fâcheux de perdre 4 textes sur 130 (perte de 3%).